In [1]:
!pip install opencv-python-headless numpy pillow requests

  Obtaining dependency information for opencv-python-headless from https://files.pythonhosted.org/packages/21/6d/abf701fa71ff22e3617ec9b46197f9ff5bba16dfefa7ee259b60216112eb/opencv_python_headless-4.8.0.76-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 31.4 MB/s eta 0:00:00:00:0100:01


In [7]:
import os
import boto3
import json
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.estimator import Estimator
import cv2
import zipfile
import numpy as np
import mxnet as mx
from mxnet import recordio
from PIL import Image
import io
import requests

In [4]:
!curl http://images.cocodataset.org/zips/train2017.zip | aws s3 cp - s3://newobjectdetectionbucket/train2017.zip
!curl http://images.cocodataset.org/zips/val2017.zip | aws s3 cp - s3://newobjectdetectionbucket/val2017.zip
!curl http://images.cocodataset.org/annotations/annotations_trainval2017.zip | aws s3 cp - s3://newobjectdetectionbucket/annotations_trainval2017.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.0G  100 18.0G    0     0  7643k      0  0:41:10  0:41:10 --:--:-- 7682k51M    0     0  6677k      0  0:47:08  0:01:09  0:45:59 7679k42:09  0:05:11  0:36:58 7718k 0  7594k      0  0:41:26  0:12:29  0:28:57 7675k  0:41:21  0:14:53  0:26:28 7670kG   41 7622M    0     0  7619k      0  0:41:18  0:17:04  0:24:14 7692k     0  7630k      0  0:41:14  0:20:00  0:21:14 7702k108M    0     0  7631k      0  0:41:14  0:20:22  0:20:52 7662k   52 9648M    0     0  7633k      0  0:41:13  0:21:34  0:19:39 7694k:41:12  0:22:34  0:18:38 7692k22:44  0:18:28 7682k0:41:12  0:23:04  0:18:08 7691k 0:41:11  0:24:40  0:16:31 7702k   68 12.2G    0     0  7646k      0  0:41:09  0:28:01  0:13:08 7654k0  7648k      0  0:41:08  0:29:37  0:11:31 7709k0:41:08  0:31:49  0:09:19 7703k:11  0:38:27  0:02:44 7714k59  0:00:11 7685k
  % Total    % Received % Xferd 

In [ ]:
role = get_execution_role()
training_image = get_image_uri(boto3.Session().region_name, 'object-detection', repo_version='latest')
s3_output_location = 's3://newobjectdetectionbucket2/traineddata/output'

In [ ]:
model = Estimator(training_image,
                  role, 
                  instance_count=1, 
                  instance_type='ml.p2.xlarge',
                  output_path=s3_output_location,
                  base_job_name='obj-detection')

In [ ]:
model.set_hyperparameters(base_network='resnet-50',
                          use_pretrained_model=1,
                          num_classes=80,
                          mini_batch_size=16,
                          epochs=30,
                          learning_rate=0.001,
                          optimizer='sgd',
                          momentum=0.9,
                          weight_decay=0.0005,
                          overlap_threshold=0.5,
                          nms_threshold=0.45,
                          image_shape=512,
                          label_width=2000,
                          num_training_samples=118287)

In [ ]:
s3train = 's3://newobjectdetectionbucket2/recordio/train2'
s3validation = 's3://newobjectdetectionbucket2/recordio/validation2'

In [ ]:
train_data = sagemaker.inputs.TrainingInput(s3train, distribution='FullyReplicated', 
                                             content_type='application/x-recordio', 
                                             s3_data_type='S3Prefix')
validation_data = sagemaker.inputs.TrainingInput(s3validation, distribution='FullyReplicated', 
                                                  content_type='application/x-recordio', 
                                                  s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}

In [ ]:
model.fit(inputs=data_channels, logs=True)

In [ ]:
predictor = model.deploy(initial_instance_count=1,
                          instance_type='ml.m4.xlarge')